In [30]:
import { ethers } from 'ethers';
const nuxtRpcUrl = Deno.env.get('NUXT_RPC_URL');
const provider = new ethers.JsonRpcProvider(nuxtRpcUrl);

In [31]:
// Contract addresses and ABIs
const UNISWAP_V3_FACTORY = "0x1F98431c8aD98523631AE4a59f267346ea31F984";
const ERC20_ABI = [
    "function decimals() view returns (uint8)",
    "function symbol() view returns (string)"
];

const FACTORY_ABI = [
    "function getPool(address tokenA, address tokenB, uint24 fee) external view returns (address pool)"
];

const POOL_ABI = [
    "function slot0() external view returns (uint160 sqrtPriceX96, int24 tick, uint16 observationIndex, uint16 observationCardinality, uint16 observationCardinalityNext, uint8 feeProtocol, bool unlocked)",
    "function token0() external view returns (address)",
    "function token1() external view returns (address)"
];

interface PoolInfo {
    SqrtX96: string;
    Pair: string;
    Decimal0: number;
    Decimal1: number;
    Token0: string;
    Token1: string;
}

const getPoolInfo = async (poolAddress: string, provider: ethers.Provider): Promise<PoolInfo> => {
    try {
        const poolContract = new ethers.Contract(poolAddress, POOL_ABI, provider);
        const [token0Address, token1Address, slot0] = await Promise.all([
            poolContract.token0(),
            poolContract.token1(),
            poolContract.slot0()
        ]);
        console.log(token0Address, token1Address, slot0);

        const [token0Contract, token1Contract] = [
            new ethers.Contract(token0Address, ERC20_ABI, provider),
            new ethers.Contract(token1Address, ERC20_ABI, provider)
        ];

        const [decimal0, decimal1, symbol0, symbol1] = await Promise.all([
            token0Contract.decimals(),
            token1Contract.decimals(),
            token0Contract.symbol(),
            token1Contract.symbol()
        ]);

        return {
            SqrtX96: slot0.sqrtPriceX96.toString(),
            Pair: `${symbol0}/${symbol1}`,
            Decimal0: decimal0,
            Decimal1: decimal1,
            Token0: token0Address,
            Token1: token1Address
        };
    } catch (error) {
        console.error("Error getting pool info:", error);
        throw error;
    }
};

interface PriceInfo {
    priceInWei: string;
    outputTokenAddress: string;
    inputTokenAddress: string;
}

export const getTokenPrice = async (
    poolAddress: string,
    outputTokenAddress: string,
    provider: ethers.Provider
): Promise<PriceInfo> => {
    try {
        const { SqrtX96, Decimal0, Decimal1, Token0, Token1 } = await getPoolInfo(poolAddress, provider);

        // Determine which token is the output token and get its decimals
        const isOutputToken0 = Token0.toLowerCase() === outputTokenAddress.toLowerCase();

        // Convert to BigNumber for precise calculations
        const sqrtPriceX96 = ethers.toBigInt(SqrtX96);
        const Q96 = ethers.toBigInt(2) ** ethers.toBigInt(96);
        const decimal0BN = ethers.toBigInt(10) ** ethers.toBigInt(Decimal0);
        const decimal1BN = ethers.toBigInt(10) ** ethers.toBigInt(Decimal1);

        // Calculate price using BigNumber
        const sqrtPrice = sqrtPriceX96 * sqrtPriceX96;
        const price = (sqrtPrice * decimal0BN) / (Q96 * Q96);

        // Calculate price in wei
        const priceInWei = isOutputToken0
            ? ((price * decimal0BN) / decimal1BN).toString()
            : ((decimal1BN * Q96 * Q96) / (sqrtPrice * decimal0BN)).toString();

        return {
            priceInWei,
            outputTokenAddress,
            inputTokenAddress: isOutputToken0 ? Token1 : Token0
        };
    } catch (error) {
        console.error("Error getting price:", error);
        throw error;
    }
};

In [32]:
const poolInfo = await getPoolInfo('0x21b8065d10f73EE2e260e5B47D3344d3Ced7596E',provider);
console.log(poolInfo);

Error getting pool info: Error: execution reverted (no data present; likely require(false) occurred (action="call", data="0x", reason="require(false)", transaction={ "data": "0x3850c7bd", "to": "0x21b8065d10f73EE2e260e5B47D3344d3Ced7596E" }, invocation=null, revert=null, code=CALL_EXCEPTION, version=6.14.3)
    at makeError (file:///Users/giorgos/workstation/personal-projects/tokenscope/node_modules/ethers/lib.esm/utils/errors.js:132:21)
    at getBuiltinCallException (file:///Users/giorgos/workstation/personal-projects/tokenscope/node_modules/ethers/lib.esm/abi/abi-coder.js:102:12)
    at AbiCoder.getBuiltinCallException (file:///Users/giorgos/workstation/personal-projects/tokenscope/node_modules/ethers/lib.esm/abi/abi-coder.js:203:16)
    at Interface.makeError (file:///Users/giorgos/workstation/personal-projects/tokenscope/node_modules/ethers/lib.esm/abi/interface.js:779:32)
    at staticCallResult (file:///Users/giorgos/workstation/personal-projects/tokenscope/node_modules/ethers/l

Error: execution reverted (no data present; likely require(false) occurred (action="call", data="0x", reason="require(false)", transaction={ "data": "0x3850c7bd", "to": "0x21b8065d10f73EE2e260e5B47D3344d3Ced7596E" }, invocation=null, revert=null, code=CALL_EXCEPTION, version=6.14.3)